In [1]:
# !pip install scikit-image

In [2]:
# !sudo apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [3]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import tensorflow as tf
import pandas as pd
from collections import namedtuple

2023-09-11 11:27:54.467673: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-11 11:27:54.492242: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 11:27:54.848530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using TensorFlow backend.


### Explore label ids

In [4]:
data = pd.read_csv("data_from_json.csv", sep='\t', encoding='utf-8', index_col=0)

In [5]:
data.head()

,label,polygon,city,pic_id,set_type,target,target_id
0,sky,"[[126, 9], [235, 182], [797, 315], [1045, 339]...",lindau,000057_000019,val,sky,5
1,road,"[[37, 561], [362, 514], [811, 398], [955, 361]...",lindau,000057_000019,val,flat,1
2,building,"[[1140, 324], [1139, 334], [1138, 337], [1123,...",lindau,000057_000019,val,construction,2
3,vegetation,"[[1129, 376], [1127, 371], [1127, 369], [1125,...",lindau,000057_000019,val,nature,4
4,building,"[[1073, 337], [1072, 294], [1055, 291], [1049,...",lindau,000057_000019,val,construction,2


In [11]:
import segmentation_models as sm

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
truc = bidule

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.python.keras.losses import binary_crossentropy
import tqdm
import cv2
from tensorflow.python.keras.callbacks import Callback

## GPU

In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [ ]:
# print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

In [ ]:
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [ ]:
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
# sess = tf.compat.v1.Session(config=config)

In [ ]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
#     except RuntimeError as e:
#         print(e)

In [ ]:
# # Ref: https://www.tensorflow.org/guide/gpu
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)

In [ ]:
# from tensorflow.compat.v1 import ConfigProto, InteractiveSession
# config = ConfigProto()
# config.gpu_options.allow_growth = True 
# sess = InteractiveSession(config=config)

## following

In [ ]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def total_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + (3*dice_loss(y_true, y_pred))
    return loss

In [ ]:
cats = {'void': [0, 1, 2, 3, 4, 5, 6],
 'flat': [7, 8, 9, 10],
 'construction': [11, 12, 13, 14, 15, 16],
 'object': [17, 18, 19, 20],
 'nature': [21, 22],
 'sky': [23],
 'human': [24, 25],
 'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]}

In [ ]:
image_dir = 'P8_Cityscapes_leftImg8bit_trainvaltest/leftImg8bit/train/aachen'
mask_dir = 'P8_Cityscapes_gtFine_trainvaltest/gtFine/train/aachen'
image_list = os.listdir(image_dir)
mask_list = os.listdir(mask_dir)
mask_list = [x for x in mask_list if x.endswith("labelIds.png")]
image_list.sort()
mask_list.sort()
print(f'. . . . .Number of images: {len(image_list)}\n. . . . .Number of masks: {len(mask_list)}')

# sanity check
# for i in range(len(image_list)):
#     assert image_list[i][16:] == mask_list[i][24:]


batch_size = 10
samples = len(image_list)
steps = samples//batch_size
img_height, img_width = 256, 256
classes = 8
filters_n = 64

In [ ]:
class seg_gen(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx = np.random.randint(0, len(image_list)+1, batch_size)
        batch_x, batch_y = [], []
        drawn = 0
        for i in idx:
            # print("AAA", len(image_list))
            # print("BBB", len(idx))
            try:
                _image = image.img_to_array(image.load_img(f'{image_dir}/{image_list[i]}', target_size=(img_height, img_width)))/255.
            except:
                print("AAA", len(image_list))
                print("BBB", len(idx))
                truc = bidule
            img = image.img_to_array(image.load_img(f'{mask_dir}/{mask_list[i]}', color_mode = "grayscale", target_size=(img_height, img_width)))
            labels = np.unique(img)
            if len(labels) < 3:
                idx = np.random.randint(0, 50000, batch_size-drawn)
                continue
            img = np.squeeze(img)
            mask = np.zeros((img.shape[0], img.shape[1], 8))
            for i in range(-1, 34):
                if i in cats['void']:
                    mask[:,:,0] = np.logical_or(mask[:,:,0],(img==i))
                elif i in cats['flat']:
                    mask[:,:,1] = np.logical_or(mask[:,:,1],(img==i))
                elif i in cats['construction']:
                    mask[:,:,2] = np.logical_or(mask[:,:,2],(img==i))
                elif i in cats['object']:
                    mask[:,:,3] = np.logical_or(mask[:,:,3],(img==i))
                elif i in cats['nature']:
                    mask[:,:,4] = np.logical_or(mask[:,:,4],(img==i))
                elif i in cats['sky']:
                    mask[:,:,5] = np.logical_or(mask[:,:,5],(img==i))
                elif i in cats['human']:
                    mask[:,:,6] = np.logical_or(mask[:,:,6],(img==i))
                elif i in cats['vehicle']:
                    mask[:,:,7] = np.logical_or(mask[:,:,7],(img==i))
            mask = np.resize(mask,(img_height*img_width, 8))
            batch_y.append(mask)
            batch_x.append(_image)
            drawn += 1
        return np.array(batch_x), np.array(batch_y)

In [ ]:
class visualize(Callback):
    def on_epoch_end(self, epoch, logs):
        print(f'\nGenerating output at epoch : {epoch}')
        i = 567
        img = image.img_to_array(image.load_img(f'{image_dir}/{image_list[i]}'))/255.    
        dims = img.shape
        x = cv2.resize(img, (256, 256))
        x = np.float32(x)/255.
        z = unet.predict(np.expand_dims(x, axis=0))
        z = np.squeeze(z)
        z = z.reshape(256, 256, 8)
        z = cv2.resize(z, (dims[1], dims[0]))
        y = np.argmax(z, axis=2)
        
        construction = np.zeros_like(y)
        human = np.zeros_like(y)
        vehicle = np.zeros_like(y)
        construction[y==2] = 255.
        human[y==6] = 255.
        vehicle[y==7] = 255.
        
        result = img.copy()
        alpha = 0.4
        img[:,:,1] = construction
        img[:,:,2] = vehicle 
        img[:,:,0] = human

        cv2.addWeighted(img, alpha, result, 1-alpha, 0, result)
        cv2.imwrite(f'outputs/{epoch}.png', cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
        print('Wrote file to disk')

In [ ]:
# train_gen = seg_gen(image_list, mask_list, batch_size)

In [ ]:
# train_gen

### DilatedNet

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout, Reshape, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import tensorflow as tf

In [ ]:
def DilatedNet(img_height, img_width, nclasses, use_ctx_module=False, bn=False):
    print('. . . . .Building DilatedNet. . . . .')
    def bilinear_upsample(image_tensor):
        # upsampled = tf.image.resize_bilinear(image_tensor, size=(img_height, img_width))
        upsampled = tf.compat.v1.image.resize_bilinear(image_tensor, size=(img_height, img_width))
        return upsampled
    
    def conv_block(conv_layers, tensor, nfilters, size=3, name='', padding='same', dilation_rate=1,pool=False):
        if dilation_rate == 1:
            conv_type = 'conv'
        else:
            conv_type = 'dilated_conv'
        for i in range(conv_layers):
            tensor = Conv2D(nfilters, size, padding=padding, use_bias=False, dilation_rate=dilation_rate, name=f'block{name}_{conv_type}{i+1}')(tensor)
            if bn:
                tensor = BatchNormalization(name=f'block{name}_bn{i+1}')(tensor)
            tensor = Activation('relu', name=f'block{name}_relu{i+1}')(tensor)
        if pool:
            tensor = MaxPooling2D(2, name=f'block{name}_pool')(tensor)
        return tensor
       
    nfilters = 64
    img_input = Input(shape=(img_height, img_width, 3))
    x = conv_block(conv_layers=2,tensor=img_input, nfilters=nfilters*1, size=3, pool=True, name=1)
    x = conv_block(conv_layers=2,tensor=x, nfilters=nfilters*2, size=3, pool=True, name=2)
    x = conv_block(conv_layers=3,tensor=x, nfilters=nfilters*4, size=3, pool=True, name=3)
    x = conv_block(conv_layers=3,tensor=x, nfilters=nfilters*8, size=3, name=4)
    x = conv_block(conv_layers=3,tensor=x, nfilters=nfilters*8, size=3,dilation_rate=2, name=5)
    x = conv_block(conv_layers=1,tensor=x, nfilters=nfilters*64, size=7,dilation_rate=4, name='_FCN1')
    x = Dropout(0.5)(x)
    x = conv_block(conv_layers=1,tensor=x, nfilters=nfilters*64, size=1, name='_FCN2')
    x = Dropout(0.5)(x)  
    x = Conv2D(filters=nclasses, kernel_size=1, padding='same', name=f'frontend_output')(x)
    if use_ctx_module:
        x = conv_block(conv_layers=2, tensor=x, nfilters=nclasses*2, size=3, name='_ctx1')
        x = conv_block(conv_layers=1, tensor=x, nfilters=nclasses*4, size=3, name='_ctx2', dilation_rate=2)
        x = conv_block(conv_layers=1, tensor=x, nfilters=nclasses*8, size=3, name='_ctx3', dilation_rate=4)
        x = conv_block(conv_layers=1, tensor=x, nfilters=nclasses*16, size=3, name='_ctx4', dilation_rate=8)
        x = conv_block(conv_layers=1, tensor=x, nfilters=nclasses*32, size=3, name='_ctx5', dilation_rate=16)        
        x = conv_block(conv_layers=1, tensor=x, nfilters=nclasses*32, size=3, name='_ctx7')
        x = Conv2D(filters=nclasses, kernel_size=1, padding='same', name=f'ctx_output')(x)
    x = Lambda(bilinear_upsample, name='bilinear_upsample')(x)
    x = Reshape((img_height*img_width, nclasses))(x)
    x = Activation('softmax', name='final_softmax')(x)
  
    model = Model(inputs=img_input, outputs=x, name='DilatedNet')
    print('. . . . .Building network successful. . . . .')
    return model

### Use DilatedNet

In [ ]:
unet = DilatedNet(256, 256, 8,use_ctx_module=True, bn=True)
# p_unet = multi_gpu_model(unet, 4)
# p_unet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coeff, 'accuracy'])
unet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coeff, 'accuracy'])
# tb = TensorBoard(log_dir='logs', write_graph=True)
mc = ModelCheckpoint(mode='max', filepath='models-dr/pdilated.h5', monitor='acc', save_best_only='True', save_weights_only='True', verbose=1)
es = EarlyStopping(mode='max', monitor='accuracy', patience=6, verbose=1)
vis = visualize()
# callbacks = [tb, mc, es]
callbacks = [mc, es]
train_gen = seg_gen(image_list, mask_list, batch_size)

In [ ]:
# # p_unet.fit_generator(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=8)
# unet.fit_generator(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=8)
unet.fit(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=8)
print('Saving final weights')
unet.save_weights('dilated.h5')

## Unet

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Activation, Dropout, Reshape, UpSampling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
# from tensorflow.python.keras.optimizers import Adadelta, Nadam
from tensorflow.python.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
# multi_gpu_model
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.python.keras.losses import binary_crossentropy

In [ ]:
def conv_block(tensor, nfilters, size=3, padding='same', initializer="he_normal"):
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [ ]:
def deconv_block(tensor, residual, nfilters, size=2, padding='same', strides=(2, 2)):
    y = UpSampling2D(size=(size, size))(tensor)
    y = Conv2D(nfilters, kernel_size=(size, size), padding=padding)(y)
    y = concatenate([y, residual], axis=3)
    y = conv_block(y, nfilters)
    return y

In [ ]:
def Unet(img_height, img_width, nclasses=3, filters=64):
# down
    input_layer = Input(shape=(img_height, img_width, 3), name='image_input')
    conv1 = conv_block(input_layer, nfilters=filters)
    conv1_out = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = conv_block(conv1_out, nfilters=filters*2)
    conv2_out = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = conv_block(conv2_out, nfilters=filters*4)
    conv3_out = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = conv_block(conv3_out, nfilters=filters*8)
    conv4_out = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv4_out = Dropout(0.5)(conv4_out)
    conv5 = conv_block(conv4_out, nfilters=filters*16)
    conv5 = Dropout(0.5)(conv5)
# up
    deconv6 = deconv_block(conv5, residual=conv4, nfilters=filters*8)
    deconv6 = Dropout(0.5)(deconv6)
    deconv7 = deconv_block(deconv6, residual=conv3, nfilters=filters*4)
    deconv7 = Dropout(0.5)(deconv7) 
    deconv8 = deconv_block(deconv7, residual=conv2, nfilters=filters*2)
    deconv9 = deconv_block(deconv8, residual=conv1, nfilters=filters)
# output
    output_layer = Conv2D(filters=nclasses, kernel_size=(1, 1))(deconv9)
    output_layer = BatchNormalization()(output_layer)
    output_layer = Reshape((img_height*img_width, nclasses), input_shape=(img_height, img_width, nclasses))(output_layer)
    output_layer = Activation('softmax')(output_layer)
    
    model = Model(inputs=input_layer, outputs=output_layer, name='Unet')
    print('Created and loaded model')
    return model

In [ ]:
unet = Unet(256, 256, nclasses=8)

In [ ]:
unet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coeff, 'accuracy'])

In [ ]:
train_gen = seg_gen(image_list, mask_list, batch_size)
unet.fit_generator(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=8)